In [1]:
!apt-get -y update

!wget -q -c -nc https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip -qq -n ngrok-stable-linux-amd64.zip

# sshの設定
!apt-get install -qq -o=Dpkg::Use-Pty=0 openssh-server pwgen > /dev/null

Hit:1 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:2 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:3 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:4 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:5 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Hit:6 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:7 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Hit:8 http://security.ubuntu.com/ubuntu bionic-security InRelease
Hit:9 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:10 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:11 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:12 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:13 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64

In [2]:
import random, string, urllib.request, json, getpass

# Generate root password
# password = ''.join(random.choice(string.ascii_letters + string.digits) for i in range(20))
password = '1qaz2wsx'

# Set root password
!echo root:$password | chpasswd
!mkdir -p /var/run/sshd
!echo "PermitRootLogin yes" >> /etc/ssh/sshd_config
!echo "PasswordAuthentication yes" >> /etc/ssh/sshd_config
!echo "LD_LIBRARY_PATH=/usr/lib64-nvidia" >> /root/.bashrc
!echo "export LD_LIBRARY_PATH" >> /root/.bashrc

# Run sshd
get_ipython().system_raw('/usr/sbin/sshd -D &')

# アクセストークンの設定
# https://dashboard.ngrok.com/auth/your-authtoken
authtoken="1wiFQHqtL3nKAxzQb7hEYINWEHU_6Eez5G7k391wBGS2qXynN"

# Create tunnel
get_ipython().system_raw('./ngrok authtoken $authtoken && ./ngrok tcp 22 &')

In [3]:
# Get public address and print connect command
with urllib.request.urlopen('http://localhost:4040/api/tunnels') as response:
  data = json.loads(response.read().decode())
  (host, port) = data['tunnels'][0]['public_url'][6:].split(':')

print(f"""Host google-colab-ssh
HostName {host}
Port {port}
User root
""")

# Print root password
print(f'Root password: {password}')

# シンボリックリンクを作成
!ln -sfn /content/drive/MyDrive/Kaggle /root/Kaggle

!cp /root/Kaggle/.setup/.gitconfig .gitconfig
!cp /root/Kaggle/.setup/.kaggle .kaggle -r


Host google-colab-ssh
HostName 0.tcp.ngrok.io
Port 13036
User root

Root password: 1qaz2wsx
